In [1]:
import os
import sys
spark_home = r"C:\spark\spark-4.0.0-bin-hadoop3"
sys.path.insert(0, spark_home + r"\python")
sys.path.insert(0, spark_home + r"\python\lib\py4j-0.10.9.9-src.zip")
os.environ["PYSPARK_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:/Users/Raghava/AppData/Local/Programs/Python/Python310/python.exe"


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag,unix_timestamp
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df_temperature = spark.read.csv("H:/leet_code_sql_50/sql_50_leetcode/sample_data/09_rising_temperature.csv", header=True, inferSchema=True)

In [5]:
df_temperature.show()

+---+----------+-----------+
| id|recorddate|temperature|
+---+----------+-----------+
|  1|2015-01-01|         10|
|  2|2015-01-02|         25|
|  3|2015-01-03|         20|
|  4|2015-01-04|         30|
+---+----------+-----------+



In [6]:
window_spec = Window.orderBy(col('recorddate'))


In [7]:

df_temperature = df_temperature.withColumn(
    "prev_temperature", lag(col('temperature')).over(window_spec)
).withColumn(
    "prev_recorddate", lag(col("recorddate")).over(window_spec)
)

In [8]:
df_temperature.show()

+---+----------+-----------+----------------+---------------+
| id|recorddate|temperature|prev_temperature|prev_recorddate|
+---+----------+-----------+----------------+---------------+
|  1|2015-01-01|         10|            NULL|           NULL|
|  2|2015-01-02|         25|              10|     2015-01-01|
|  3|2015-01-03|         20|              25|     2015-01-02|
|  4|2015-01-04|         30|              20|     2015-01-03|
+---+----------+-----------+----------------+---------------+



In [12]:
df_temperature.where(
    (col('temperature') > col('prev_temperature')) &
    ((unix_timestamp(col('recorddate')) - unix_timestamp(col('prev_recorddate'))) == 86400)  # 86400 seconds = 1 day
).select('id').show()

+---+
| id|
+---+
|  2|
|  4|
+---+

